In [69]:
#Basic working model using collaborative filtering with watched, this is just removing the watched movies from list

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [49]:

# Sample user data
user_data = {
    'id': [1, 2, 3],
    'happy_movie': [['Drama', 'Comedy'], ['Romance', 'Family'], ['Adventure']],
    'sad_movie': [['Drama', 'Thriller'], ['Crime'], ['Horror', 'Science Fiction']],
    'neutral_movie': [['Comedy'], ['Action', 'Adventure'], ['Crime']],
    'fav_movie': ['The Godfather', "Schindler's List", 'The Shawshank Redemption']
}
user = pd.DataFrame(user_data)


# Load movie data from the specified file
file_path = '/content/drive/MyDrive/final project sample data/10000 Movies Data'
minfo = pd.read_csv(file_path)
movie = minfo[['Movie_id', 'title', 'Genres', 'popularity', 'vote_average', 'vote_count']]



# Sample watched data
watched_data = {
    'userid': [1, 2, 3, 1, 2],
    'movieid': [238, 278, 240, 19404, 122],
    'rating_given': [8.0, 5.5, 9.5, 7.0, 6.5],
    'watching_mood': ['Happy', 'Sad', 'Neutral', 'Happy', 'Neutral']
}
watched = pd.DataFrame(watched_data)

In [50]:
movie

,Movie_id,title,Genres,popularity,vote_average,vote_count
0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",93.552,8.7,16814
1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",78.664,8.7,22542
2,240,The Godfather Part II,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",55.752,8.6,10187
3,19404,Dilwale Dulhania Le Jayenge,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",22.150,8.6,3927
4,424,Schindler's List,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",53.542,8.6,13382
...,...,...,...,...,...,...
9975,384737,Precious Cargo,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",14.663,5.2,243
9976,282813,The Pyramid,"[{'id': 27, 'name': 'Horror'}]",17.892,5.2,861
9977,134375,Home Alone: The Holiday Heist,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",23.978,5.2,335
9978,10477,Driven,"[{'id': 28, 'name': 'Action'}]",13.200,5.2,594


In [51]:
# prompt: Using dataframe movie: i want Genres to be reformatted with only list of name of genres, right now it has id and name and addtional text

import ast

# Create a copy of the dataframe
movie_genres = movie.copy()

# Convert the Genres column from string to list of dictionaries
movie_genres['Genres'] = movie_genres['Genres'].apply(ast.literal_eval)

# Extract the names of the genres from the list of dictionaries
movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [i['name'] for i in x])

# Display the first 10 rows of the dataframe
movie_genres.head(10)


,Movie_id,title,Genres,popularity,vote_average,vote_count
0,238,The Godfather,"[Drama, Crime]",93.552,8.7,16814
1,278,The Shawshank Redemption,"[Drama, Crime]",78.664,8.7,22542
2,240,The Godfather Part II,"[Drama, Crime]",55.752,8.6,10187
3,19404,Dilwale Dulhania Le Jayenge,"[Comedy, Drama, Romance]",22.150,8.6,3927
4,424,Schindler's List,"[Drama, History, War]",53.542,8.6,13382
5,129,Spirited Away,"[Animation, Family, Fantasy]",78.519,8.5,13476
6,667257,Impossible Things,"[Family, Drama]",10.676,8.5,271
7,389,12 Angry Men,[Drama],27.767,8.5,6754
8,372058,Your Name.,"[Romance, Animation, Drama]",120.614,8.5,9178
9,496243,Parasite,"[Comedy, Thriller, Drama]",61.872,8.5,14608


In [52]:
# Assuming user, watched, and movie DataFrames are already defined as per your description.

# Merge the watched movies data with the movie details
watched_movies = pd.merge(watched, movie_genres, left_on='movieid', right_on='Movie_id')



In [53]:
watched_movies

,userid,movieid,rating_given,watching_mood,Movie_id,title,Genres,popularity,vote_average,vote_count
0,1,238,8.0,Happy,238,The Godfather,"[Drama, Crime]",93.552,8.7,16814
1,2,278,5.5,Sad,278,The Shawshank Redemption,"[Drama, Crime]",78.664,8.7,22542
2,3,240,9.5,Neutral,240,The Godfather Part II,"[Drama, Crime]",55.752,8.6,10187
3,1,19404,7.0,Happy,19404,Dilwale Dulhania Le Jayenge,"[Comedy, Drama, Romance]",22.150,8.6,3927
4,2,122,6.5,Neutral,122,The Lord of the Rings: The Return of the King,"[Adventure, Fantasy, Action]",119.972,8.5,20423


In [54]:

from itertools import chain

# Assuming 'user' is your DataFrame containing user preferences.
# Sample data structure for 'user' DataFrame is provided in your initial description.

# Function to expand and tag mood to genres
def expand_user_genres(user_df, mood_column):
    # Explode the DataFrame by the specified mood to handle multiple genres
    expanded_df = user_df[['id', mood_column]].explode(mood_column)
    # Assign the current mood as a new column
    expanded_df['mood'] = mood_column.split('_')[0].capitalize()
    # Rename the genre column to a generic 'genre' column
    expanded_df = expanded_df.rename(columns={mood_column: 'genre'})
    return expanded_df

# Process each mood
happy_genres = expand_user_genres(user, 'happy_movie')
sad_genres = expand_user_genres(user, 'sad_movie')
neutral_genres = expand_user_genres(user, 'neutral_movie')

# Concatenate all moods together
all_genres = pd.concat([happy_genres, sad_genres, neutral_genres])

# Optional: Reset index if you want a clean DataFrame index
all_genres.reset_index(drop=True, inplace=True)

all_genres.head()


,id,genre,mood
0,1,Drama,Happy
1,1,Comedy,Happy
2,2,Romance,Happy
3,2,Family,Happy
4,3,Adventure,Happy


In [55]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming movie_genres DataFrame has a 'Genres' column with lists of genres
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres['Movie_id'])



In [56]:
def recommend_movies_with_genres_and_history(user_id, current_mood, top_n=5):
    # Filter user preferences for the current mood
    mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
    user_pref_genres = mood_genres['genre'].unique()

    # Encode user preferences
    user_pref_encoded = mlb.transform([user_pref_genres])

    # Calculate similarity between user preferences and each movie's genres
    similarity = cosine_similarity(user_pref_encoded, movie_genres_encoded)

    # Get watched movie IDs to exclude them from recommendations
    watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()

    # Filter out watched movies from the recommendations
    potential_movies = movie_genres[~movie_genres.index.isin(watched_movie_ids)]
    potential_movie_ids = potential_movies.index.values
    filtered_similarity = similarity[:, potential_movie_ids]

    # Get top N recommended movie IDs based on similarity scores, excluding watched movies
    recommended_movie_indices = filtered_similarity.argsort()[0][-top_n:][::-1]
    recommended_movie_ids = potential_movies.iloc[recommended_movie_indices].index

    # Ensure the index of movie_genres matches Movie_id and check data type consistency
    recommended_movie_ids = recommended_movie_ids.astype(movie_genres.index.dtype)  # Adjust if necessary

    # Map movie IDs back to movie titles and genres
    recommended_movies_info = movie_genres.loc[recommended_movie_ids][['title', 'Genres']]

    # Create a more readable output format
    recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1)

    return recommended_movies_output.tolist()


# Example usage, recommending movies for user 1 in a "Happy" mood, considering their watching history
recommended_movies = recommend_movies_with_genres_and_history(1, 'Happy', 5)
for movie in recommended_movies:
    print(movie)


Happiness (Genres: Comedy, Drama)
Slap Shot (Genres: Comedy, Drama)
Welcome Home Roscoe Jenkins (Genres: Comedy, Drama)
Skins (Genres: Drama, Comedy)
This Is Where I Leave You (Genres: Drama, Comedy)


In [57]:
# Example: Recommend movies for user 1 in a "Happy" mood, including genres
recommended_movies = recommend_movies_with_genres(1, 'Happy', 5)
for movie in recommended_movies:
    print(movie)

Ingrid Goes West (Genres: Comedy, Drama)
Jenny's Wedding (Genres: Comedy, Drama)
Boy (Genres: Drama, Comedy)
Diner (Genres: Comedy, Drama)
The Toy (Genres: Comedy, Drama)


In [58]:
recommended_movies = recommend_movies_with_genres(1, 'Sad', 5)
for movie in recommended_movies:
    print(movie)

Cold in July (Genres: Drama, Thriller)
The Nightingale (Genres: Drama, Thriller)
The Truth About Emanuel (Genres: Drama, Thriller)
New Order (Genres: Thriller, Drama)
Derailed (Genres: Drama, Thriller)


In [59]:
recommended_movies = recommend_movies_with_genres(1, 'Neutral', 5)
for movie in recommended_movies:
    print(movie)

The ComDads (Genres: Comedy)
Caddyshack (Genres: Comedy)
I Can Quit Whenever I Want (Genres: Comedy)
Blended (Genres: Comedy)
Billy Madison (Genres: Comedy)


In [60]:
recommended_movies = recommend_movies_with_genres(2, 'Happy', 5)
for movie in recommended_movies:
    print(movie)


A Christmas Prince: The Royal Baby (Genres: Romance, Family)
The Holiday Calendar (Genres: Romance, Comedy, Family)
Keith (Genres: Drama, Family, Romance)
The Princess Diaries (Genres: Comedy, Family, Romance)
Miss Potter (Genres: Drama, Family, Romance)


In [61]:
recommended_movies = recommend_movies_with_genres(2, 'Sad', 5)
for movie in recommended_movies:
    print(movie)

The Good Liar (Genres: Crime)
The Sicilian Clan (Genres: Crime)
Oh Mercy (Genres: Crime)
Extremely Wicked, Shockingly Evil and Vile (Genres: Crime)
The Beast (Genres: Crime)


In [62]:
recommended_movies = recommend_movies_with_genres(2, 'Neutral', 5)
for movie in recommended_movies:
    print(movie)

Indiana Jones and the Last Crusade (Genres: Adventure, Action)
12 Rounds 2: Reloaded (Genres: Action, Adventure)
Mission: Impossible - Fallout (Genres: Action, Adventure)
Cutthroat Island (Genres: Action, Adventure)
Raiders of the Lost Ark (Genres: Adventure, Action)


In [63]:
recommended_movies = recommend_movies_with_genres(3, 'Happy', 5)
for movie in recommended_movies:
    print(movie)

The Flight of the Phoenix (Genres: Adventure)
Belle and Sebastian (Genres: Adventure)
The Water Man (Genres: Adventure)
Young Sherlock Holmes (Genres: Adventure)
Ben 10 Alien Swarm (Genres: Adventure)


In [64]:
recommended_movies = recommend_movies_with_genres(3, 'Sad', 5)
for movie in recommended_movies:
    print(movie)

Dr. Jekyll and Mr. Hyde (Genres: Horror, Science Fiction)
Screamers (Genres: Horror, Science Fiction)
Chopping Mall (Genres: Horror, Science Fiction)
C.H.U.D. (Genres: Horror, Science Fiction)
The Curse of Frankenstein (Genres: Horror, Science Fiction)


In [65]:
recommended_movies = recommend_movies_with_genres(3, 'Neutral', 5)
for movie in recommended_movies:
    print(movie)

The Good Liar (Genres: Crime)
The Sicilian Clan (Genres: Crime)
Oh Mercy (Genres: Crime)
Extremely Wicked, Shockingly Evil and Vile (Genres: Crime)
The Beast (Genres: Crime)


In [66]:
user

,id,happy_movie,sad_movie,neutral_movie,fav_movie
0,1,"[Drama, Comedy]","[Drama, Thriller]",[Comedy],The Godfather
1,2,"[Romance, Family]",[Crime],"[Action, Adventure]",Schindler's List
2,3,[Adventure],"[Horror, Science Fiction]",[Crime],The Shawshank Redemption


Relevant code ends here

In [67]:
minfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9980 non-null   int64  
 1   Movie_id      9980 non-null   int64  
 2   title         9980 non-null   object 
 3   Genres        9980 non-null   object 
 4   release_date  9980 non-null   object 
 5   Keywords      9980 non-null   object 
 6   overview      9974 non-null   object 
 7   poster_path   9980 non-null   object 
 8   Budget        9980 non-null   int64  
 9   Revenue       9980 non-null   int64  
 10  popularity    9980 non-null   float64
 11  vote_average  9980 non-null   float64
 12  vote_count    9980 non-null   int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1013.7+ KB


In [68]:
minfo

,Unnamed: 0,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count
0,0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1972-03-14,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814
1,1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1994-09-23,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542
2,2,240,The Godfather Part II,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1974-12-20,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...",In the continuing saga of the Corleone crime f...,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,102600000,102600000,55.752,8.6,10187
3,3,19404,Dilwale Dulhania Le Jayenge,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-10-19,[],"Raj is a rich, carefree, happy-go-lucky second...",/2CAL2433ZeIihfX1Hb2139CX0pW.jpg,100000000,100000000,22.150,8.6,3927
4,4,424,Schindler's List,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",1993-12-15,"[{'id': 818, 'name': 'based on novel or book'}...",The true story of how businessman Oskar Schind...,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,321365567,321365567,53.542,8.6,13382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,9975,384737,Precious Cargo,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",2016-07-15,"[{'id': 9730, 'name': 'crime boss'}, {'id': 10...","After a botched heist, Eddie a murderous crim...",/dOQv2PDBu9bBCZRLeRp6QQYJDr3.jpg,100659,100659,14.663,5.2,243
9976,9976,282813,The Pyramid,"[{'id': 27, 'name': 'Horror'}]",2014-12-04,"[{'id': 1299, 'name': 'monster'}, {'id': 3762,...",An archaeological team attempt to unlock the s...,/joPb4PUPP7xClD6XPiQ7laBEtZf.jpg,16861137,16861137,17.892,5.2,861
9977,9977,134375,Home Alone: The Holiday Heist,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",2012-11-25,"[{'id': 207317, 'name': 'christmas'}]",8-year-old Finn is terrified to learn his fami...,/6JPrRC0JPM06y17pUXD6w1xMvKi.jpg,0,0,23.978,5.2,335
9978,9978,10477,Driven,"[{'id': 28, 'name': 'Action'}]",2001-04-27,"[{'id': 271, 'name': 'competition'}, {'id': 36...",Talented rookie race-car driver Jimmy Bly has ...,/2aN2BBAbRyfEpfxZAQY82T8zWRq.jpg,54744738,54744738,13.200,5.2,594
